In [1]:
import numpy as np

### Functions

In [2]:
def prep_lines(lines):
    for i in range(len(lines)):
        lines[i]=lines[i].replace('     * ', '')
        lines[i]=lines[i].replace(' \n', '')
        lines[i]=lines[i].replace('\n', '')
    return lines

'''
Read fortran ODEs and prep for translation.
    - file = file name
    - returns: dict
'''
def read_fortran_ODEs(file):

    ## read fortran ODEs from given file
    with open(file, 'r') as f:
        lines = f.readlines()

    ## select non-conserved species
    spec_lines = lines[123:]

    ## remove unnecessary fortran code
    spec_lines = prep_lines(spec_lines)

    ## make dict:
    ##      - entry per spec
    ##      - stick together separate fortran strings
    ODEs = dict()

    for i, line in enumerate(spec_lines):
        if len(line) != 0 and line[0] == 'C':
            ODEs[line[2:]] = str('')
            print('i=',i, end = '\r')
            start = i + 1

            for j in range(start,start+1000):
                if j >= len(spec_lines):
                    break
                if len(spec_lines[j]) != 0 and spec_lines[j][0] == 'C':
                    stop = j
                    break
                
            for j in range(i+1,stop):
                ODEs[line[2:]] += spec_lines[j]

            i = stop +1

    del ODEs['']

    return ODEs


'''
Translate fortran ODE to python ODE
'''
def fortran_to_python(ODEs, spec):
    print(spec)

    if isinstance(ODEs[spec],str):
        ## NOT fractional abundances, but in units of cm^-3
        ODEs[spec] = ODEs[spec].replace('*HNR', '')
        ## split string
        ODEs[spec] = ODEs[spec].split(' ')

    ## remove empty strings
    to_remove = ''
    while to_remove in ODEs[spec]:
        ODEs[spec].remove('')

    ## remove other fortran commands
    while 'END' in ODEs[spec]:
        ODEs[spec].remove('END')
    while 'RETURN' in ODEs[spec]:
        ODEs[spec].remove('RETURN')
    
    ## replace brackets: ( --> [
    for i in range(len(ODEs[spec])):
        # print(i,ODEs[spec][i][0])
        if ODEs[spec][i][0] == 'F' or ODEs[spec][i][0] == 'D':
            # print('yes!')
            ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
            ODEs[spec][i] = ODEs[spec][i].replace(')', ']')

    ## replace final line
    last = ODEs[spec][-1]
    # print('last', last)
    while not last[0] == 'Y':
        ODEs[spec].remove(last)
        last = ODEs[spec][-1]
    # print('at the end',last)
    for i in range(5,len(last)):
        if last[i] == ')':
            stop = i
            break
    nb = last[5:stop]

    ODEs[spec][-1] = 'YDOT['+nb+']=F-(D*Y['+nb+'])'

    return ODEs
    

### Read in file & prep

In [3]:
file = '/lhome/silkem/CHEM/src-IP-AP-HNR/code/acodes.f'

with open(file, 'r') as f:
    lines = f.readlines()

ODEs = read_fortran_ODEs(file)


In [4]:
ODEs['C11+']

'      F=0.+K(1517)*Y(10)*Y(458)*HNR+K(1518)*Y(10)*Y(455)*HNR+K(2066)*Y(11)*Y(453)*HNR+K(2068)*Y(11)*Y(459)*HNR+K(5727)*Y(465)+K(6042)*Y(10)*Y(451)*HNR+K(6202)*Y(465)+K(6538)*Y(465)      D=0.+K(994)*X(1)*HNR+K(995)*X(1)*HNR+K(996)*X(1)*HNR+K(997)*X(1)*HNR      YDOT(466)=F-(D*Y(466))      RETURN      END'

### Translate

In [5]:
for key in ODEs:
    ODEs = fortran_to_python(ODEs, key)

H
H+
H-
H2+
H3+
He
He+
HeH+
C-
C+
C
CH-
CH+
CH
CH2+
N+
CH2
N
CH3
NH
NH+
CH3+
O+
NH2+
CH4+
O-
NH2
O
CH4
OH-
OH
OH+
CH5+
NH3+
NH3
NH4+
H2O
H2O+
F
H3O+
F+
HF
HF+
H2F+
Na
Na+
C2-
C2+
Mg
Mg+
C2
C2H-
C2H+
C2H
CN
C2H2
C2H2+
CN-
CN+
HCN
C2H3
C2H3+
HCN+
HNC
N2
C2H4+
Si+
H2CN
H2NC+
N2+
Si
CO
CO+
C2H4
HCNH+
C2H5+
HCO
SiH+
HCO+
CH2NH
SiH
C2H5
HOC+
N2H+
CH2NH2+
SiH2
CH3CH3+
NO
H2CO
CH4N+
H2CO+
CH3CH3
SiH2+
NO+
P+
P
H3CO+
HNO+
SiH3+
HNO
CF+
C2H7+
SiH3
O2
SiH4
PH
S
H2NO+
CH3OH
SiH4+
O2+
S-
O2-
PH+
CH3OH+
S+
CH3OH2+
O2H
PH2+
HS
HS+
PH2
O2H+
SiH5+
H2O2
H2S+
H2S
PH3+
H3S+
Cl
Cl+
C3+
HCl
C3-
C3
HCl+
H2Cl+
C3H-
C3H+
C3H
C2N
C3H2+
C2N+
H2CCC
CNC+
C3H2
C2NH+
C3H3+
CH2CCH
CH2CCH+
SiC
CH2CCH2
SiC+
C2O
C3H4+
C2O+
CH2CN
CH3CCH
CH2CN+
C3H5+
HCSi
HCSi+
CH3CN+
HC2O+
CH3CN
C3H6+
SiN
CH3CNH+
NH2CN
CH3CHCH2
SiN+
SiCH2+
SiCH2
CNO
CH2CO+
OCN
OCN+
CH2CO
CP+
HNCO
HCNO+
HNCO+
HONC
SiCH3+
HCNO
CP
C3H7+
NH2CNH+
HNSi+
HOCN
HNSi
CH3CO+
HOCN+
SiCH3
HONC+
CS+
HCNOH+
N2O+
CS
SiCH4+
CO2+
H2OCN+
HNCOH+
SiNH2+
HCP
SiO+
CH3CHO
SiO
H

In [6]:
ODEs['C11+']

['F=0.+K[1517]*Y[10]*Y[458]+K[1518]*Y[10]*Y[455]+K[2066]*Y[11]*Y[453]+K[2068]*Y[11]*Y[459]+K[5727]*Y[465]+K[6042]*Y[10]*Y[451]+K[6202]*Y[465]+K[6538]*Y[465]',
 'D=0.+K[994]*X[1]+K[995]*X[1]+K[996]*X[1]+K[997]*X[1]',
 'YDOT[466]=F-(D*Y[466])']

### Write out in as python function